![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `ibm/granite-3-3-8b-instruct` to perform chat conversation with JSON response format

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook provides a detailed demonstration of the steps and code required to showcase support for JSON response format in Chat models.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The purpose of this notebook is to demonstrate how to use JSON response format in Chat models and how to specify the response JSON schema.

## Table of Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [Work with JSON response format](#json-response-format)
- [Work with specified JSON schema response format](#json-specific-format)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependencies

In [1]:
%pip install -U "ibm-watsonx-ai" | tail -n 1

### Define the watsonx.ai credentials
Use the code cell below to define the watsonx.ai credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [2]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your watsonx.ai api key and hit enter: "),
)

### Define the project ID
You need to provide the project ID to give the Foundation Model the context for the call. If you have a default project ID set in Watson Studio, the notebook obtains that project ID. Otherwise, you need to provide the project ID in the code cell below.

In [3]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your project_id and hit enter: ")

<a id="models"></a>
## Set up the Foundation Model on `watsonx.ai`


Specify the `model_id` of the model you will use for the chat with tools.

In [4]:
model_id = "ibm/granite-3-3-8b-instruct"

### Model parameters overview

In order to receive the response from the model in the JSON format, the `TextChatResponseFormatType.JSON_OBJECT` response format must be specified. You might also need to adjust model parameters depending on the model you use.

In [5]:
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters

TextChatParameters.show()

+-----------------------+----------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| PARAMETER             | TYPE                                   | EXAMPLE VALUE                                                                                                                                                                                                                                                                   |
+=======================+========================================+============================================================================================================================================================================================================================================================

In [6]:
from ibm_watsonx_ai.foundation_models.schema import (
    TextChatResponseFormat,
    TextChatResponseFormatType,
)

TextChatResponseFormat.show()

+-------------+--------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| PARAMETER   | TYPE                                       | EXAMPLE VALUE                                                                                                                                                                                                                           |
+=============+============================================+=========================================================================================================================================================================================================================================+
| type        | str, TextChatResponseFormatType            | json_schema                                           

<a id="json-response-format"></a>
## Work with JSON response format

### Initialize the model

Initialize the `ModelInference` class with provided parameters.

In [7]:
from ibm_watsonx_ai.foundation_models import ModelInference


params = TextChatParameters(
    response_format=TextChatResponseFormat(TextChatResponseFormatType.JSON_OBJECT),
    max_tokens=1024,
    temperature=1,
)

model = ModelInference(
    model_id=model_id, credentials=credentials, project_id=project_id, params=params
)

### Create messages and chat with the model

In order to ensure the response is in the correct format, the sent messages must contain an indication that JSON is expected.

In [8]:
messages = [
    {"role": "system", "content": "Respond in a JSON format"},
    {"role": "user", "content": "Describe methods of calculating pi"},
]

chat_response = model.chat(messages=messages, params=params)

### Parse the response

Use the `json` library to parse the chat response content into a Python-native data structure.

In [9]:
import json

json_response_content = json.loads(chat_response["choices"][0]["message"]["content"])
print(json.dumps(json_response_content, ensure_ascii=False, indent=2))

{
  "methods": [
    {
      "method": "Gregory-Leibniz Series",
      "description": "This is an infinite series which alternately adds and subtracts successive terms of a harmonic sequence. It converges very slowly, so it's not practical for calculating many digits of pi. The formula is pi = 4 * (1 - 1/3 + 1/5 - 1/7 + 1/9 - ...)"
    },
    {
      "method": "Nilakantha Series",
      "description": "Also known as the Nilakantha infinite series or the Mountains and valleys series, this method converges much faster than the Gregory-Leibniz series. The formula is pi = 3 + 4/(2*3*4) - 4/(4*5*6) + 4/(6*7*8) - 4/(8*9*10) ..."
    },
    {
      "method": "Machin-like formulae",
      "description": "These formulae were discovered by John Machin and improved by others. They are—pi/4 = 4*arctan(1/5) - arctan(1/239) (the convergence here is particularly fast due to the arctan(1/239) term)."
    },
    {
      "method": "Gauss-Legendre Algorithm",
      "description": "This is an algorithm fo

<a id="json-specific-format"></a>
## Work with specified JSON schema response format

### Initialize the model

Initialize a new `ModelInference` class with the provided parameters, including the expected JSON schema. For more info about JSON schema, visit: https://json-schema.org/learn

In [10]:
params = TextChatParameters(
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "Cake recipes",
            "schema": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "cake_name": {
                            "type": "string",
                        },
                        "description": {
                            "type": "string",
                        },
                        "difficulty_score": {
                            "type": "integer",
                        },
                        "expected_price": {
                            "type": "number",
                        },
                    },
                },
            },
            "strict": True,
        },
    }
)

model = ModelInference(
    model_id=model_id, credentials=credentials, project_id=project_id, params=params
)

### Create messages and chat with the model

As previously, in order to ensure the response is in the correct format, the sent message must contain an indication that JSON is expected.

In [11]:
messages = [
    {"role": "system", "content": "Respond in a JSON format."},
    {
        "role": "user",
        "content": "Provide a list of cake recipes. Briefly describe what the cake tastes like. Give each a difficulty score between 1-10. Also add an expected price of the cake with accuracy of 2 decimal places.",
    },
]

chat_response = model.chat(messages=messages)

### Parse the response

As previously, use the `json` library to parse the chat response content into a Python-native data structure.

In [12]:
json_response_content = json.loads(chat_response["choices"][0]["message"]["content"])
print(json.dumps(json_response_content, ensure_ascii=False, indent=2))

[
  {
    "cake_name": "Vanilla Cake",
    "description": "Light, fluffy, and naturally sweet with a subtle vanilla flavor.",
    "difficulty_score": 3,
    "expected_price": 25.99
  },
  {
    "cake_name": "Chocolate Fudge Cake",
    "description": "Rich, decadent, and deeply chocolatey with a creamy fudge center.",
    "difficulty_score": 5,
    "expected_price": 32.5
  },
  {
    "cake_name": "Red Velvet Cake",
    "description": "Slightly tangy and sweet with a distinctive red color and cream cheese frosting.",
    "difficulty_score": 6,
    "expected_price": 35.0
  },
  {
    "cake_name": "Lemon Drizzle Cake",
    "description": "Zesty and tangy from the lemon, balanced with a light, moist sponge.",
    "difficulty_score": 4,
    "expected_price": 28.75
  },
  {
    "cake_name": "Marble Cake",
    "description": "A visually attractive cake with a swirl of choclate batter within vanilla.",
    "difficulty_score": 4,
    "expected_price": 30.0
  },
  {
    "cake_name": "Carrot Cake"

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to work with chat models using tools and watsonx.ai.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Rafał Chrzanowski**, Software Engineer Intern at watsonx.ai.

Copyright © 2025 IBM. This notebook and its source code are released under the terms of the MIT License.